# GEMdataset

> This package will hold the GxExM dataset.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
#| hide
from hybridpredictmaize22.GEMdataset import *

In [ ]:
#| export
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
#| hide
def remove_leapdays(weather):
    """ just a hotfix """
    to_remove = []
    for i in list(set(weather_data['Env'])):
        if (sum(weather_data['Env'] == i)) == 366:
            #get indexes
            to_remove.append(max(list(weather_data.loc[weather_data['Env'] == i].index)))
    return weather_data.drop(to_remove)

In [ ]:
ARGS = {
'test_split':2019,
'path_snps':Path('data/snpCompress/'),
'data_path':Path('data/'),
'path_train_weatherTable':Path('data/Training_Data/4_Training_Weather_Data_2014_2021.csv'),
'path_train_yieldTable':Path('data/Training_Data/1_Training_Trait_Data_2014_2021.csv'),
'snp_compression':'PCS_50',
'batch_size':64
}

In [ ]:
test_split = 2019
path_snps = Path('data/snpCompress/')
data_path = Path('data/')
path_train_weatherTable =data_path/'Training_Data/4_Training_Weather_Data_2014_2021.csv'
path_train_yieldTable = data_path/'Training_Data/1_Training_Trait_Data_2014_2021.csv'
snp_compression = 'PCS_50'
batch_size = 64

def setup_data():
    
    snp_data = collect_snps(path_snps/snp_compression) # Read in the SNP profiles
    yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
    weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
    
    yield_data = yield_data[yield_data['Twt_kg_m3'].notnull()] #Remove plots w/ missing yields
    #yield_data = yield_data.reset_index()
    weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
    #removes yield data where no weather data
    setYield = set(yield_data['Env'])
    setWeather = set(weather_data['Env'])
    only_yield = setYield - setWeather
    only_weather = setWeather - setYield
    yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
    #removes yield data where no genotype data
    setSNP = set(snp_data[0])
    setYield = set(yield_data['Hybrid'])
    only_yield = setYield - setSNP
    yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

    weather_data = remove_leapdays(weather_data)
    #weather_data = weather_data.reset_index()
    #yield_data=yield_data.sample(frac=1)
    yield_data = yield_data.reset_index()
    
    #Create a GEM dataset
    gem = GEM(test_split)
    gem.Y = YT(yield_data, test_split)
    gem.W = WT(weather_data, test_split)
    gem.SNP = snp_data
    
    tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
    te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)
    
    tr_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
    te_dataloader = DataLoader(te_ds, batch_size=batch_size, shuffle=False)
    
    return tr_dataloader, te_dataloader

In [ ]:
snp_data = collect_snps(path_snps/snp_compression) # Read in the SNP profiles
yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
yield_data = yield_data[yield_data['Twt_kg_m3'].notnull()] #Remove plots w/ missing yields
weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
#removes yield data where no weather data
setYield = set(yield_data['Env'])
setWeather = set(weather_data['Env'])
only_yield = setYield - setWeather
only_weather = setWeather - setYield
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
#removes yield data where no genotype data
setSNP = set(snp_data[0])
setYield = set(yield_data['Hybrid'])
only_yield = setYield - setSNP
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

weather_data = remove_leapdays(weather_data)
weather_data = weather_data.reset_index()
#yield_data=yield_data.sample(frac=1)
yield_data = yield_data.reset_index()

#Create a GEM dataset
gem = GEM(test_split)

gem.Y = YT(yield_data, test_split)
gem.W = WT(weather_data, test_split)
gem.SNP = snp_data

tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)

tr_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
te_dataloader = DataLoader(te_ds, batch_size=batch_size, shuffle=True)

In [ ]:
def init_oz(layer):
    inSize = layer.in_features
    outSize = layer.out_features
    layer.weight = torch.nn.Parameter(torch.tensor(np.array(np.random.choice([-1, 0, 1], size=(outSize,inSize)))).type(torch.float32))
    return layer

In [ ]:
class gModel(nn.Module):
    def __init__(self, inSide,hiddenSize):
        super().__init__()
        #self.in = nn.Linear()
        self.inputLayer = init_oz(nn.Linear(hiddenSize,inSide))
        self.hiddenLayer = init_oz(nn.Linear(hiddenSize,1))

    def forward(self,x):
        x = self.inputLayer(x)
        print(x)
        return x

In [ ]:

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = init_oz(nn.Linear(input_size, hidden_size))
        self.relu = nn.ReLU()
        self.fc2 = init_oz(nn.Linear(hidden_size, output_size))

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    
    
    
# Create an instance of the MLP class


In [ ]:
def unhook(t):
    return t.detach().numpy()

In [ ]:
class DataLoaders:
    def __init__(self, dls):
        #print(dls)
        self.train,self.valid = dls[:2]
        

In [ ]:
#|export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [ ]:
#|export
class Callback(): order = 0

In [ ]:
#|export
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        #method_nm ({before,after} {batch,epoch,fit})
        method = getattr(cb, method_nm, None)
        if method is not None: method(learn)

In [ ]:
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func=torch.optim.SGD): fc.store_attr()

    def one_batch(self):        
        #my edit
        self.yb, self.gb, self.wb = [to_device(x).type(torch.float32) for x in self.batch]
        
        self.preds = self.model(self.gb).squeeze()
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()

    def one_epoch(self, train):
        self.model.training = train
        self.dl = self.dls.train if train else self.dls.valid
        try:
            self.callback('before_epoch')
            for self.iter,self.batch in enumerate(self.dl):
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException: pass
            self.callback('after_epoch')
        except CancelEpochException: pass
        
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try:
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException: pass
        
        #note the self here refers to the Learner object of the class
    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)
        
    def __getattr__(self, name):
        if name in ('predict','get_loss','backward','step','zero_grad'): return partial(self.callback, name)
        raise AttributeError(name)

In [ ]:
def to_device(x): return x.to('cpu')

In [ ]:
class StopBatch(Callback):
    def before_fit(self,learn): self.count=0
    def after_batch(self,learn):
        self.count+=1
        if self.count > 10:
            raise CancelEpochException()
            
            
    def after_epoch(self,learn): pass
    def after_fit(self, learn): print('toast')

In [ ]:
import fastcore.all as fc
from operator import attrgetter,itemgetter

model = MLP(input_size=100, hidden_size=2, output_size=1)#opt = Optimizer(gw.parameters(),lr=0.001)
opt = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_func = nn.MSELoss()
dls = setup_data()
lr=.3


In [ ]:

x = Learner(model, DataLoaders(dls), loss_func, lr,  cbs=[CompletionCB(), StopBatch()])

In [ ]:
def_device = 'cpu'
x.fit(1)

Completed 12 batches
toast


In [ ]:
class Metric:
    def __init__(self): self.reset()
    def reset(self): self.vals,self.ns = [],[]
    def add(self, inp, targ=None, n=1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    def calc(self, inps, targs): return inps

In [ ]:
#|export
from torcheval.metrics import MeanSquaredError, Mean

In [ ]:
metric = MeanSquaredError()
metric.update(torch.tensor([0, 2, 1, 3]), torch.tensor([0, 1, 2, 3]))
metric.compute().sqrt()
metric.reset()
metric.compute()

tensor(nan)

In [ ]:
#|export
from copy import copy
import torch.nn.functional as F
from collections.abc import Mapping

class MetricsCB(Callback):
    
    def __init__(self, *ms, **metrics):
            #this block just unpacks the init args
        for o in ms: metrics[type(o).__name__] = o
        self.metrics = metrics
        self.all_metrics = copy(metrics)
        self.all_metrics['loss'] = self.loss = Mean()

    def _log(self, d): print(d)
    def before_fit(self, learn): learn.metrics = self
    def before_epoch(self, learn): [o.reset() for o in self.all_metrics.values()]

    def after_epoch(self, learn):
        log = {k:f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
        log['epoch'] = learn.epoch
        log['train'] = 'train' if learn.model.training else 'eval'
        self._log(log)

    def after_batch(self, learn):
        y,g,w = to_cpu(learn.batch)
        for m in self.metrics.values(): m.update(to_cpu(learn.preds), y)
        self.loss.update(to_cpu(learn.loss), weight=len(y))

In [ ]:
metrics = MetricsCB(accuracy=MeanSquaredError())
#|export
class DeviceCB(Callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self, learn): learn.model.to(self.device)
    def before_batch(self, learn): learn.batch = to_device(learn.batch, device=self.device)

In [ ]:
#|export
def to_cpu(x):
    if isinstance(x, Mapping): return {k:to_cpu(v) for k,v in x.items()}
    if isinstance(x, list): return [to_cpu(o) for o in x]
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    return x.detach().cpu()

In [ ]:
model = MLP(input_size=100, hidden_size=2, output_size=1)#opt = Optimizer(gw.parameters(),lr=0.001)
opt = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_func = nn.MSELoss()
metrics = MetricsCB(accuracy=MeanSquaredError())
learn = Learner(model, DataLoaders(dls), F.cross_entropy, lr=0.2, cbs=[metrics])
learn.fit(1)

{'accuracy': '207.336', 'loss': '-0.619', 'epoch': 0, 'train': 'train'}
{'accuracy': '0.565', 'loss': '-25.647', 'epoch': 0, 'train': 'eval'}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()